# Library

In [1]:
import time
import tensorflow as tf
import os
import cv2
import numpy as np

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [2]:
pwd

'C:\\Users\\B00330\\Desktop\\face_mask_detection'

# Load Model from Checkpoint

In [34]:
custom_model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PATH_TO_MODEL_DIR = os.path.join('workspace', 'training_demo', 'models', custom_model_name)

# PATH_TO_CFG = PATH_TO_MODEL_DIR + "/{}.config".format(custom_model_name)
PATH_TO_CFG = os.path.join(PATH_TO_MODEL_DIR, 'inference_graph', 'pipeline.config')
# PATH_TO_CKPT = PATH_TO_MODEL_DIR + "/checkpoint"
# print(PATH_TO_CFG)

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(r"C:\Users\B00330\Desktop\face_mask_detection\workspace\training_demo\models\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\inference_graph\pipeline.config")
# configs = config_util.get_configs_from_pipeline_file('/Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection/workspace/training_demo/models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config')
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_MODEL_DIR, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model... Done! Took 0.06119346618652344 seconds


In [35]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

# Load model

In [20]:
pwd

'/Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection'

In [31]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

custom_model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PATH_TO_MODEL_DIR = os.path.join('workspace', 'training_demo', 'models', custom_model_name, 'inference_graph')
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 8.58633279800415 seconds


In [32]:
PATH_TO_LABELS = os.path.join('workspace', 'training_demo', 'annotations', 'label_map.pbtxt')
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

# Real Time Detection

In [33]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    # input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break